# CH3 Decision Tree

## Introductions

http://en.wikipedia.org/wiki/ID3_algorithm
- 决策树学习通常包含三个步骤: 特征选择，决策树生成，决策树修剪[Lihang P55]
- 决策树模型
    - if-then规则
    - 条件概率分布

信息定义 $$l(x_i)=-log_2p(x_i)$$
熵定义 $$H=-\sum_{i=1}^{n}p(x_i)log_2p(x_i)$$
决策树学习的损失函数通常是正则化的极大似然函数。

决策树的学习策略是以损失函数为目标函数的最小化。

### Tree Map Demo

In [ ]:
from pyecharts import TreeMap

data = [
    {
        "value": 50,
        "name": "发送邮件域名地址为:🔗🔗🔗🔗:是->无聊时需要阅读",
    },
    {
        "value": 50,
        "name": "发送邮件域名地址为:🔗🔗🔗🔗:否",
        "children": [
            {"value": 50,
             "name": "是否包含单词曲棍球:是->需要及时处理的朋友邮件",
             },
            {
                "value": 50,
                "name": "是否包含单词曲棍球:否->垃圾邮件",
            },

        ]
    },

]

treemap = TreeMap("决策树","可以通过下钻的方式展示决策的过程，例子参考MLiA中图3-1", width=800, height=500)
treemap.use_theme("dark")
treemap.add("发送邮件域名地址为:🔗🔗🔗🔗", data, is_label_show=True, label_pos='inside', treemap_left_depth=1)
treemap

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Feature Importance

Refer to [1]

## MLiA

### Create Dataset

In [ ]:
dataSet = pd.DataFrame({'no surfacing':[1,1,1,0,0],'flippers':[1,1,0,1,1],'is Fish':['yes','yes','no','no','no']},columns=["no surfacing","flippers","is Fish"])
dataSet

### ID3

#### Metric

##### Entropy

In [ ]:
len(dataSet)

In [ ]:
def calcShannonEnt(dataSet):
    col = dataSet.columns.tolist()[-1]
    prob = dataSet.groupby(by=col).size()/len(dataSet)
    return sum(-np.log2(prob)*prob)    

In [ ]:
calcShannonEnt(dataSet)

In [ ]:
dataSet["is Fish"].iloc[0] = "maybe"

In [ ]:
dataSet

In [ ]:
calcShannonEnt(dataSet)

##### Conditional Entroy

书里少解释了一个点，条件熵(conditional entropy)

$$ H(Y|X)=\sum_{i=1}^{n}p_iH(Y|X=x_i)$$

其中$p_i=P(X=x_i),i=1,2,...,n$

[李航，2012，P61]

#### Partition

In [ ]:
def splitDataSet(dataSet,axis,value):
    return dataSet[dataSet[axis] == value][dataSet.columns.drop(axis)]

In [ ]:
dataSet

In [ ]:
splitDataSet(dataSet,"flippers",1)

In [ ]:
calcShannonEnt(splitDataSet(dataSet,"no surfacing",0))

In [ ]:
calcShannonEnt(splitDataSet(dataSet,"no surfacing",1))

In [ ]:
calcShannonEnt(splitDataSet(dataSet,"flippers",0))

In [ ]:
calcShannonEnt(splitDataSet(dataSet,"flippers",1))

#### Feature Selection

In [ ]:
def chooseBestFeatureToSplit(dataSet):
    feas = dataSet.columns.tolist()[:-1]
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0
    
    for fea in feas:
        newEntropy = 0
        for value in set(dataSet[fea]):
            subSet = splitDataSet(dataSet,fea,value)
            #conditional entropy
            Pi = len(subSet)/len(dataSet)
            newEntropy += Pi*calcShannonEnt(subSet)
    
        infoGain = baseEntropy - newEntropy
        if  (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = fea
    return bestFeature

In [ ]:
calcShannonEnt(dataSet)

In [ ]:
dataSet.columns

In [ ]:
feas = dataSet.columns.tolist()[:-1]
feas

In [ ]:
set(dataSet["flippers"])

In [ ]:
chooseBestFeatureToSplit(dataSet)

### Tree

#### Create Tree

In [ ]:
dataSet

In [ ]:
import operator
def majorityCnt(dataSet):
    label = dataSet.columns.tolist()[-1]
    return sorted(dataSet.groupby(by=label).size().iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]
majorityCnt(dataSet)

In [ ]:
# iteritems
list(dataSet.groupby(by="is Fish").size().iteritems())

In [ ]:
def createTree(dataSet):
#     print("======")
#     print(dataSet)
    # stop condition
    feas = dataSet.columns.tolist()[:-1]
    label = dataSet[dataSet.columns.tolist()[-1]]
    # 参考下李航 CH5 里面算法的描述
    # entropy == 0
    if len(set(label)) == 1:
#         print("condition 1")
        return list(label)[0]
    # 参考下李航CH5里面的算法描述
    # no feature to use
    if len(set(feas)) == 0:
#         print("condition 2")
        return majorityCnt(dataSet)
    bestFeature = chooseBestFeatureToSplit(dataSet)
    myTree = {bestFeature:{}}
    featValues = set(dataSet[bestFeature])
#     print(featValues)
    for value in featValues:
        subDataSet = dataSet[dataSet[bestFeature]==value]
        myTree[bestFeature][value] = createTree(splitDataSet(subDataSet,bestFeature,value))
    return myTree

In [ ]:
dataSet

In [ ]:
myTree = createTree(dataSet)
myTree

#### Plot Tree

In [ ]:
decisionNode = dict(boxstyle="sawtooth",fc="0.8")
leafNode = dict(boxstyle="round4",fc="0.8")
arrow_args= dict(arrowstyle="<-")
def plotNode(nodeTxt, centerPt, parentPt, nodeType):
    createPlot.ax1.annotate(nodeTxt,rotation = 90,
                            xy=parentPt,xycoords="axes fraction",
                            xytext=centerPt,textcoords="axes fraction",
                            va="center",ha="center",bbox=nodeType,arrowprops=arrow_args)
def createPlot():
    fig = plt.figure(1, facecolor="white")
    fig.clf()
    createPlot.ax1 = plt.subplot(111,frameon=False)
    plotNode('a decision node',(0.5,0.1),(0.1,0.5),decisionNode)
    plotNode('a leaf node',(0.8,0.1),(0.3,0.8),leafNode)
    plt.show()
createPlot()

In [ ]:
myTree.keys()

In [ ]:
myTree.items()

In [ ]:
dataSet[dataSet["is Fish"] == "maybe"].index.tolist()

In [ ]:
myTree.values()

In [ ]:
labels = ["no surfacing","flippers"]
labels.index("flippers")

In [ ]:
myTree

In [ ]:
list(myTree.keys())[0]

In [ ]:
"flippers" in myTree.keys()

In [ ]:
myTree

In [ ]:
def checkNode(nodeIn):
    keys = list(nodeIn.keys())
    for key in keys:
        childNode = nodeIn[key]
        print("key",key)
        if type(childNode).__name__ == 'dict':
            checkNode(childNode)
        else:
            print("leaf:",childNode)

In [ ]:
checkNode(myTree)

In [ ]:
print(list(myTree.keys())[0])

In [ ]:
def getNumLeafs(myTree):
    numLeafs = 0
    firstStr = list(myTree.keys())[0]
    secondDict = myTree[firstStr]
    for key in secondDict.keys():
        if type(secondDict[key]).__name__ == "dict":
            numLeafs += getNumLeafs(secondDict[key])
        else:
            numLeafs += 1
    return numLeafs

def getTreeDepth(myTree):
    maxDepth = 0
    firstStr = list(myTree.keys())[0]
    secondDict = myTree[firstStr]
    for key in secondDict.keys():
        if type(secondDict[key]).__name__=="dict":
            thisDepth = 1 + getTreeDepth(secondDict[key])
        else:
            thisDepth = 1
        if thisDepth > maxDepth: maxDepth = thisDepth
    return maxDepth

In [ ]:
def plotMidText(cntrPt, parentPt, txtString):
    xMid = (parentPt[0]-cntrPt[0])/2.0 + cntrPt[0]
    yMid = (parentPt[1]-cntrPt[1])/2.0 + cntrPt[1]
    createPlot.ax1.text(xMid,yMid,txtString)
# 参数命名尽量不要和全局的重合，不方便debug
def plotTree(myTree, parentPt, nodeTxt):
    numLeafs = getNumLeafs(myTree)
    depth = getTreeDepth(myTree)
#     print("dict",myTree)
    firstStr = list(myTree.keys())[0]
    cntrPt = (plotTree.xOff + (1.0 + float(numLeafs))/2.0/plotTree.totalW, plotTree.yOff)
    plotMidText(cntrPt, parentPt, nodeTxt)
    plotNode(firstStr,cntrPt,parentPt,decisionNode)
    secondDict = myTree[firstStr]
    plotTree.yOff = plotTree.yOff - 1.0/plotTree.totalD
#     print("second",secondDict)
    for key in secondDict.keys():
        if type(secondDict[key]).__name__ == 'dict':
#             print("key",key,"dict",secondDict[key])
            plotTree(secondDict[key],cntrPt,str(key))
        else:
            plotTree.xOff = plotTree.xOff + 1.0/plotTree.totalW
            plotNode(secondDict[key],(plotTree.xOff,plotTree.yOff),cntrPt,leafNode)
            plotMidText((plotTree.xOff,plotTree.yOff),cntrPt,str(key))
    plotTree.yOff = plotTree.yOff + 1.0/plotTree.totalD

def createPlot(inTree):
    fig = plt.figure(1,facecolor='white')
    fig.clf()
    axprops = dict(xticks=[],yticks=[])
    createPlot.ax1 = plt.subplot(111,frameon=False, **axprops)
    plotTree.totalW = float(getNumLeafs(inTree))
    plotTree.totalD = float(getTreeDepth(inTree))
    plotTree.xOff = -0.5/plotTree.totalW
    plotTree.yOff = 1.0
    plotTree(inTree,(0.5,1.0),'')
    plt.show()

In [ ]:
createPlot(myTree)

In [ ]:
myTree

In [ ]:
def tmpfun():
    return tmpfun.test

In [ ]:
tmpfun.test = 1
tmpfun()

### Classifier

#### Classify

In [ ]:
def classify(inputTree, featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == "dict":
                classLabel = classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
    return classLabel       

In [ ]:
myTree

In [ ]:
classify(inputTree=myTree,featLabels=["no surfacing","flippers"],testVec=[1,1])

#### Store

In [ ]:
def storeTree(inputTree,filename):
    import pickle
    with open(filename,'wb') as f:
        pickle.dump(inputTree,f)
def grabTree(filename):
    import pickle
    with open(filename,'rb') as f:
        return pickle.load(f)

#### Lenses

##### Data

In [ ]:
lenseData = pd.read_csv("./Data/CH3/lenses/lenses.data",
                index_col=0,
                names=["idx","age of the patient","spectacle prescription","astigmatic","tear production rate","type"],
                sep = "\s+")
# note this sep

In [ ]:
lenseData.head()

##### Decision Tree

In [ ]:
lenseTree = createTree(lenseData)
lenseTree

In [ ]:
createPlot(lenseTree)

In [ ]:
from pyecharts import TreeMap

data = [
    {
        "value": 50,
        "name": "reduced->no lenses",
    },
    {
        "value": 50,
        "name": "normal->astigmatic",
        "children": [
            {"value": 50,
             "name": "yes->prescript",
             "children":[
                 {"value":50,
                  "name": "hyper->age",
                  "children":[
                      {"value":30,
                      "name":"pre->nolenses"},
                      {"value":30,
                      "name":"presbyopic->nolenses"},
                      {"value":30,
                      "name":"young->hard"}
                  ]
                  },
                 {
                     "value":50,
                     "name": "myope->hard",
                 },
             ]
             },
            {
                "value": 50,
                "name": "no->age",
                "children":[
                          {"value":30,
                           "name":"pre->soft"},
                          {"value":30,
                           "name":"presbyopic->prescript",
                       "children":[
                                  {"value":50,
                                  "name":"hyper->soft"},
                                  {"value":50,
                                  "name":"myope->no lenses"},
                       ]
                      },
                      {"value":30,
                       "name":"young->soft"}
                ]
            },

        ]
    },

]

treemap = TreeMap("隐形眼镜分类决策树","", width=800, height=500)
treemap.use_theme("dark")
treemap.add("tearRate", data, is_label_show=True, label_pos='inside')
treemap

## Sklearn

### Fish

#### Load Data

In [ ]:
import pandas as pd

In [ ]:
dataSet = pd.DataFrame({'no surfacing':[1,1,1,0,0],'flippers':[1,1,0,1,1],'is Fish':['yes','yes','no','no','no']},columns=["no surfacing","flippers","is Fish"])
dataSet

#### Classify

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_fish = DecisionTreeClassifier(criterion="entropy",)

In [ ]:
X = dataSet.values[:,:-1]
y = dataSet.values[:,-1]
dt_fish.fit(X,y)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz 
dot_data = export_graphviz(dt_fish,out_file=None,
                           feature_names=dataSet.columns.tolist()[:-1],
                           class_names=["no","yes"],
                           filled=True,
                           rounded=True,  
                           special_characters=True) 
graph = graphviz.Source(dot_data)
graph

以上
- 和图3-2是一样的，0.5划分0，1，因为恰巧这个例子是二分类，还是0，1
- 书中是按照ID3，sklearn按照CART,参考[相关文档](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms)
- 这个例子gini和entropy没区别

### Lenses

#### Load Data

In [ ]:
lenseData = pd.read_csv("./Data/CH3/lenses/lenses.data",
                index_col=0,
                names=["idx","age of the patient","spectacle prescription","astigmatic","tear production rate","type"],
                sep = "\s+")
lenseData.head()

In [ ]:
lenseData.describe()

In [ ]:
X = lenseData.values[:,:-1]
y = lenseData.values[:,-1]

#### Classify

In [ ]:
dt_lenses = DecisionTreeClassifier(criterion="gini")

In [ ]:
dt_lenses.fit(X,y)

In [ ]:
dot_data = export_graphviz(dt_lenses,out_file=None,
                           feature_names=lenseData.columns.tolist()[:-1],
                           class_names=["hard","soft","no lenses"],
                           filled=True,
                           rounded=True,  
                           special_characters=True)
graph = graphviz.Source(dot_data)
graph

以上
- Sklearn实现了优化的CART，是二叉树

What are all the various decision tree algorithms and how do they differ from each other? Which one is implemented in scikit-learn?

ID3 (Iterative Dichotomiser 3) was developed in 1986 by Ross Quinlan. The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.

C4.5 is the successor to ID3 and removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. These accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.

C5.0 is Quinlan’s latest version release under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.

CART (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.

**scikit-learn uses an optimised version of the CART algorithm**

## Random Forest

## References

   [1] [L. Breiman, and A. Cutler, "Random Forests"](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm)